In [1]:
import os.path
import mne
import numpy as np
import pandas as pd

from utils.dataloader import load_subject_labels

wd = '/Volumes/Guillaume EEG Project'
epoch_data_path = os.path.join(wd, 'Berlin_Data/EEG/preprocessed/stim_epochs_incl_response')
raw_data_path = os.path.join(wd, 'Berlin_Data/EEG/raw')

In [5]:
# Load the raw data
subject_id = 204

raws = []
for file in [os.path.join(wd, raw_data_path, '{}/{}_2afc_run{}.vhdr'.format(subject_id, subject_id, suffix)) for suffix in range(1, 7)]:
    if os.path.isfile(file):
        raw = mne.io.read_raw_brainvision(file)
        raws.append(raw)

behav_data = load_subject_labels(os.path.join(wd, raw_data_path), subject_id=subject_id)

print(behav_data)

Extracting parameters from /Volumes/Guillaume EEG Project/Berlin_Data/EEG/raw/204/204_2afc_run1.vhdr...
Setting channel info structure...
Extracting parameters from /Volumes/Guillaume EEG Project/Berlin_Data/EEG/raw/204/204_2afc_run2.vhdr...
Setting channel info structure...
Extracting parameters from /Volumes/Guillaume EEG Project/Berlin_Data/EEG/raw/204/204_2afc_run3.vhdr...
Setting channel info structure...
Extracting parameters from /Volumes/Guillaume EEG Project/Berlin_Data/EEG/raw/204/204_2afc_run4.vhdr...
Setting channel info structure...
Extracting parameters from /Volumes/Guillaume EEG Project/Berlin_Data/EEG/raw/204/204_2afc_run5.vhdr...
Setting channel info structure...
Extracting parameters from /Volumes/Guillaume EEG Project/Berlin_Data/EEG/raw/204/204_2afc_run6.vhdr...
Setting channel info structure...
     response  confidence  choice_rt  correct  session  run
0          -1           2   0.616309        1        1    5
1          -1           1   0.868505        1       

In [6]:
## SPLIT RAW INTO BLOCKS
# Specify the annotation type you want to use for splitting
split_annotation_type = 'Stimulus/S105'

raw_segments = []
for raw in raws:
    # Find the onset times of the relevant annotations
    split_onsets = [annot['onset'] for annot in raw.annotations if annot['description'] == split_annotation_type]

    # Sort the onsets and add the start and end times of the raw object for splitting
    split_onsets = split_onsets + [raw.times[-1]]

    # Iterate over consecutive pairs of split points to create new raw segments
    for start, end in zip(split_onsets[:-1], split_onsets[1:]):
        raw_segment = raw.copy().crop(tmin=start, tmax=end, include_tmax=False)
        trial_n = pd.Series(raw_segment.annotations.description).value_counts()['Stimulus/S151']
        if trial_n == 55:
            raw_segments.append(raw_segment)

# Now raw_segments is a list of mne.io.Raw objects split at the specified annotations
print(len(raw_segments))

for i, seg in enumerate(raw_segments):
    print('---------------{}---------------'.format(i+1))
    print(seg)
    print(pd.Series(seg.annotations.description).value_counts())
    trial_n = pd.Series(seg.annotations.description).value_counts()['Stimulus/S151']
    print(seg.info['meas_date'])
    print('{} trials'.format(trial_n))
    print('')

6
---------------1---------------
<RawBrainVision | 204_2afc_run1.eeg, 63 x 423445 (423.4 s), ~93 kB, data not loaded>
Stimulus/S 50    550
Stimulus/S150     55
Stimulus/S 64     55
Stimulus/S 48     55
Stimulus/S 46     55
Stimulus/S 56     55
Stimulus/S 73     55
Stimulus/S151     55
Stimulus/S 11     45
Stimulus/S 29     45
Stimulus/S 40     29
Stimulus/S  6     28
Stimulus/S  8     28
Stimulus/S  7     28
Stimulus/S 41     26
Stimulus/S  3     24
Stimulus/S  4     24
Stimulus/S  5     24
Stimulus/S 20     11
Stimulus/S 28     10
Stimulus/S 10     10
Stimulus/S 76      8
Stimulus/S 21      8
Stimulus/S 22      7
Stimulus/S 23      7
Stimulus/S105      1
Name: count, dtype: int64
2023-07-27 14:08:06.526241+00:00
55 trials

---------------2---------------
<RawBrainVision | 204_2afc_run2.eeg, 63 x 394707 (394.7 s), ~93 kB, data not loaded>
Stimulus/S 50    550
Stimulus/S150     55
Stimulus/S 64     55
Stimulus/S 48     55
Stimulus/S 46     55
Stimulus/S 56     55
Stimulus/S 73     55
S

In [6]:
# TODO: will need to select correct segments and correct behavioural data.
#       Don't know if this will work in an automated fashion, otherwise compare everything by eye...
raw_segments = raw_segments[1:]
print(len(raw_segments))

8


In [4]:
## CREATE NEW ANNOTATIONS FOR RESPONSE TIMES
for i, seg in enumerate(raw_segments):
    df = behav_data[behav_data['run'] == i + 1]

    # Step 1: Extract the onsets of 'Stimulus/S 48' annotations
    stimulus_onsets = [annot['onset'] for annot in seg.annotations if annot['description'] == 'Stimulus/S 48']

    # Step 2: Ensure the number of onsets matches the number of rows in the DataFrame
    if len(stimulus_onsets) != len(df):
        raise ValueError("The number of 'Stimulus/S 48' annotations does not match the number of rows in the DataFrame.")

    # Step 3: Create new annotations based on the reaction times in the DataFrame
    for onset, choice_rt in zip(stimulus_onsets, df['choice_rt']):
        late_threshold = 1.000
        if not np.isnan(choice_rt):
            new_onset = onset + choice_rt  # Calculate the new annotation onset time
            if choice_rt < late_threshold:
                seg.annotations.append(onset=new_onset, duration=0, description='Response')
            else:
                seg.annotations.append(onset=new_onset, duration=0, description='Late Response')
        else:
            new_onset = onset + 0.001  # Calculate the new annotation onset time
            seg.annotations.append(onset=new_onset, duration=0, description='No Response')

119


ValueError: The number of 'Stimulus/S 48' annotations does not match the number of rows in the DataFrame.

In [6]:
# for i, an in enumerate(raw_segments[-1].annotations):
#     if an['description'] in [*['Stimulus/S{:>3}'.format(stim) for stim in [20, 21, 22, 23, 48]], 'Response', 'No Response']:
#         print(an)
#     if i == 200:
#         break

In [8]:
## EXTRACT EPOCHS
# Define the epochs
tmin = -1.0  # Start of each epoch (e.g., 2250 ms before the event)
tmax = 2.5   # End of each epoch (e.g., 250 ms after the event)

epochs_list = []
for seg in raw_segments:
    events, event_id = mne.events_from_annotations(seg, event_id={'Stimulus/S 64': 1}) #event_id={'Response': 1, 'Late Response': 2, 'No Response': 3})
    epochs_list.append(mne.Epochs(seg, events, event_id=event_id, tmin=tmin, tmax=tmax, baseline=None, preload=True))

# Step 5: Inspect the epochs
print(epochs_list[0])

Used Annotations descriptions: [np.str_('Stimulus/S 64')]
Not setting metadata
55 matching events found
No baseline correction applied
0 projection items activated
Loading data for 55 events and 3501 original time points ...
0 bad epochs dropped
Used Annotations descriptions: [np.str_('Stimulus/S 64')]
Not setting metadata
55 matching events found
No baseline correction applied
0 projection items activated
Loading data for 55 events and 3501 original time points ...
1 bad epochs dropped
Used Annotations descriptions: [np.str_('Stimulus/S 64')]
Not setting metadata
55 matching events found
No baseline correction applied
0 projection items activated
Loading data for 55 events and 3501 original time points ...
0 bad epochs dropped
Used Annotations descriptions: [np.str_('Stimulus/S 64')]
Not setting metadata
55 matching events found
No baseline correction applied
0 projection items activated
Loading data for 55 events and 3501 original time points ...
0 bad epochs dropped
Used Annotations

In [ ]:
# TODO: select epochs that have response, drop the rest according to annotation 'Late Response' / 'No Response'.